This is a DEMO for loading a pre-trained eqpolarity models from both California and Texas

Make sure eqpolarity is installed following the instructions on the main website
https://github.com/chenyk1990/eqpolarity

In [1]:
## Import data
#polall is polarity label from Texas (Size: 22980x1)
#datall is 600-sample waveform data (Z-component) from Texas (Size: 22980 x 600 x 1); the waveform is centered by P-arrival (manual from Texas analysts) sample

import numpy as np

# datall_Texas.npy can also be downloaded from https://mega.nz/file/chxx1Z5Y#zXNRKT5aeNy7AGREKEUIq71TREK8hcUyXA1ZOkQ9DlM
# datall = np.load('../data/TexasData/datall_Texas.npy') 
polall = np.load('../data/TexasData/polall_Texas.npy')

data=[]
for ii in range(6):
    data.append(np.load('../data/TexasData/datall_Texas%d.npy'%(ii+1)))
datall=np.concatenate(data,axis=0)


FileNotFoundError: [Errno 2] No such file or directory: '../data/TexasData/polall_Texas.npy'

In [ ]:
## Load EQpolarity model
from eqpolarity.utils import construct_model
input_shape = (600,1)
model=construct_model(input_shape)
model.summary()

In [ ]:
## Load pre-trained model for prediction
model.load_weights('../models/best_weigths_Binary_SCSN_Best.h5')
out = model.predict(datall,batch_size=1024, verbose=1)

In [ ]:
## Applying threshold
#outtest = np.argmax(out,axis=-1)
thre = 0.5
outtest = out
outtest[outtest<thre]=0
outtest[outtest>=thre]=1
labtest = polall

## Calculating accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_score(labtest,outtest),precision_score(labtest,outtest, average='micro'),recall_score(labtest,outtest, average='micro'),f1_score(labtest,outtest, average='micro')

In [ ]:
accuracy_score(labtest,outtest),precision_score(labtest,outtest, average=None),recall_score(labtest,outtest, average=None),f1_score(labtest,outtest, average=None)

In [ ]:
#Generate the confusion matrix
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(labtest, outtest)
print(cf_matrix)

In [ ]:
## plotting confusion matrix
from eqpolarity import plot_confusionmatrix
plot_confusionmatrix(cf=cf_matrix,categories=['Up','Down'],figname='Conf_Matrix_before_transferlearning.png',ifshow=True)

In [ ]:
model.load_weights("../models/best_weigths_Binary_Texas_Transfer10.h5")
out = model.predict(datall,batch_size=1024, verbose=1)

In [ ]:
#outtest = np.argmax(out,axis=-1)
thre = 0.5
outtest = out
outtest[outtest<thre]=0
outtest[outtest>=thre]=1
labtest = polall

In [ ]:
## plotting different types of accuracy
accuracy_score(labtest,outtest),precision_score(labtest,outtest, average='micro'),recall_score(labtest,outtest, average='micro'),f1_score(labtest,outtest, average='micro')

In [ ]:
accuracy_score(labtest,outtest),precision_score(labtest,outtest, average=None),recall_score(labtest,outtest, average=None),f1_score(labtest,outtest, average=None)

In [ ]:
accuracy_score(labtest,outtest),precision_score(labtest,outtest, average='macro'),recall_score(labtest,outtest, average='macro'),f1_score(labtest,outtest, average='macro')

In [ ]:
cf_matrix = confusion_matrix(labtest, outtest)
print(cf_matrix)



In [ ]:
## plot confusion matrix
from eqpolarity import plot_confusionmatrix
plot_confusionmatrix(cf=cf_matrix,categories=['Up','Down'],figname='Conf_Matrix_after_transferlearning.png',ifshow=True)